## Ayudantía 4: Threads 🧵🧶

### Ayudantes 👾
- Sección 1: [Julián García](https://github.com/JJJGGGG)
- Sección 2: [Clemente Campos](https://github.com/mskdancers)
- Sección 3: [Diego Toledo](https://github.com/diegoftpxd)
- Sección 4: [Julio Huerta](https://github.com/Julius9)
- Sección 5: [Carlos Olguín](https://github.com/CarlangaUC)

### 📖 Contenidos 📖
En esta ayudantía usaremos:
- _Threads_ para generar concurrencia
- _Locks_ para proteger zonas críticas
- _Events_ para comunicar _Threads_
### Introducción
Los Threads son una herramienta indispensable en la programación, ya que nos permiten generar concurrencia entre distintas partes del código. Esto quiere decir que vamos a poder hacer que dos partes distintas de nuestro código se ejecuten de forma simultánea. 
### ¿Cómo puedo crear un thread?

In [19]:
import threading  # esta es la librería que contiene los threads

# establecemos una función que queremos que ejecute al activarse
def funcion_objetivo(): 
    for iteracion in range(10):
        print(f"hola {iteracion}")

# instanciamos un Thread y le damos una función objetvo (sin paréntesis)
thread = threading.Thread(target=funcion_objetivo) 
thread.start()

hola 0
hola 1
hola 2
hola 3
hola 4
hola 5
hola 6
hola 7
hola 8
hola 9


### ¿Cómo puedo coordinar distintos threads?
Los threads no se ejecutan de forma perfectamente simultánea, sólo simulan que lo hacen, por lo que un thread en cualquier momento se puede pausar para que se ejecute otro. Esto hace que se puedan descoordinar generándonos problemas. Sin embargo, como programadores tenemos muchas herramientas para asegurarnos que nuestro código haga exactamente lo que queremos 🤓.
###  Join()
Para que un thread espere a que otro termine de ejecutarse, podemos usar la función ```join()```. Si es que llamamos al método ```join()``` de ```thread_1``` en la ejecución de ```thread_2```, entonces ```thread_2``` esperará a que se termine de ejecutar el ```thread_1``` para seguir su ejecución. Al método le podemos pasar como argumento el número de segundos máximo que esperará al otro thread (timeout), y pasados esos segundos seguirá su ejecución. En el siguiente ejemplo, el thread principal está esperando a que termine de ejecutarse ```thread_a_esperar```, con un timeout de tres segundos.

In [20]:
import threading
import time


def saludar():
    for iteracion in range(5):
        time.sleep(1)
        print(f"Buenos días {iteracion + 1}")

thread_a_esperar = threading.Thread(target=saludar)
thread_a_esperar.start()

# esperará tres segundos o a que termine el thread, lo que pase primero
thread_a_esperar.join(3) 
print("Acabo de perder la paciencia *muy enojado*")

# al no especificar timeout se queda esperando hasta que termine el thread
thread_a_esperar.join() 

print("Ya terminaron todos los saludos *se calma*")

Buenos días 1
Buenos días 2
Acabo de perder la paciencia *muy enojado*
Buenos días 3
Buenos días 4
Buenos días 5
Ya terminaron todos los saludos *se calma*


### is_alive()
Este método nos entrega ```True``` si el thread sigue activo o ```False``` si ya terminó su ejecución.

In [21]:
import threading
import time

def saludar():
    for iteracion in range(5):
        time.sleep(2)
        print(f"Buenos días {iteracion + 1}")

thread_a_esperar = threading.Thread(target=saludar)
thread_a_esperar.start()

while True:
    is_alive = thread_a_esperar.is_alive()
    print(f"¿el thread está vivo? {is_alive}")
    if not is_alive:
        print("Se ha terminado, nooooo")
        break
    time.sleep(1)
    


¿el thread está vivo? True
¿el thread está vivo? True
Buenos días 1¿el thread está vivo? True

¿el thread está vivo? True
Buenos días 2
¿el thread está vivo? True
¿el thread está vivo? True
Buenos días 3
¿el thread está vivo? True
¿el thread está vivo? True
Buenos días 4
¿el thread está vivo? True
¿el thread está vivo? True
Buenos días 5
¿el thread está vivo? False
Se ha terminado, nooooo


En este último ejemplo podemos ver que los prints están medios raros 🤔, sin embargo podemos arreglar esto con el uso de:

### Locks
Lo que pasa es que los dos threads intentan imprimir cosas en la consola, y como en cualquier momento uno se puede pausar para que siga el otro, nuestros prints se pueden superponer. La consola es una zona crítica y por tanto debemos poner un ```Lock``` a esta parte.
Un ```lock``` hace que sólo el thread que tenga el ```Lock``` en ese momento pueda estar interactuando con cierta parte del codigo. Si un thread toma el ```Lock```, todos los demás van a esperar a que lo suelte para poder entrar a esa parte del código.

In [22]:
import threading
import time

lock_consola = threading.Lock() # creamos el lock

def saludar():
    for iteracion in range(5):
        time.sleep(2)
        # mientras esté dentro del bloque indentado, este thread tiene el lock
        # si es que llega a esta línea y no lo tiene, espera a que lo suelten
        with lock_consola:
            print(f"Buenos días {iteracion + 1}")

thread_a_esperar = threading.Thread(target = saludar)
thread_a_esperar.start()

while True:
    is_alive = thread_a_esperar.is_alive()
    time.sleep(1)
    # mientras esté dentro del bloque indentado, este thread tiene el lock
    # si es que llega a esta línea y no lo tiene, espera a que lo suelten
    with lock_consola:
        print(f"¿el thread está funcionando? {is_alive}")
        if not(is_alive):
            print("Se ha terminado, nooooo")
            break
    

¿el thread está funcionando? True
Buenos días 1
¿el thread está funcionando? True
¿el thread está funcionando? True
Buenos días 2
¿el thread está funcionando? True
¿el thread está funcionando? True
Buenos días 3
¿el thread está funcionando? True
¿el thread está funcionando? True
Buenos días 4
¿el thread está funcionando? True
¿el thread está funcionando? True
Buenos días 5
¿el thread está funcionando? True
¿el thread está funcionando? False
Se ha terminado, nooooo


Es recomendable poner locks en todas las zonas críticas de nuestro código, las más comunes son:
* Variables globales (compartidas por varios threads)
* Archivos que se leen y escriben por varios threads
* Prints de consola
### Event
Los eventos son una forma de comunicar threads, un evento tiene una _flag_ interna que puede estar activada o apagada y puede ser vista por todos los threads. Con esto en mente, podemos usar eventos para controlar y coordinar threads.
* ```evento.wait()```: el thread que lo ejecute va a quedarse esperando hasta que la flag interna del evento sea activada.
* ```evento.set()```: activa la flag interna del evento para que todos los threads lo puedan ver.
* ```evento.clear()```: desactiva la flag interna.
* ```evento.is_set()```: entrega ```True``` si la flag está activa y ```False``` sino, pero no se queda esperando.

In [23]:
import threading
import time

lock_consola = threading.Lock()
# creamos una instancia de evento
evento_despierto = threading.Event() 

def ciclo_de_sueño():
    # duerme por 6 segundos
    for iteracion in range(6):
        time.sleep(1)
        with lock_consola:
            print("Amigo: zZzZzZzZz")

    # vamos a simular que despierta
    with lock_consola:
        # levantamos la flag interna del evento
        evento_despierto.set() 
        print("Amigo: Me acabo de despertar")

    for iteracion in range(3):
        time.sleep(1)
        with lock_consola:
            print("Amigo: *jugando*")

thread_a_esperar = threading.Thread(target=ciclo_de_sueño)
thread_a_esperar.start()

with lock_consola:
    print("Voy a esperar que mi amigo despierte")

# esperamos a que se levante la flag, por lo que este thread se queda parado
evento_despierto.wait()

with lock_consola:
    print("Entonces vamos a jugas :)")    

for iteracion in range(3):
    time.sleep(1)
    with lock_consola:
        print("*jugando*")
    

Voy a esperar que mi amigo despierte
Amigo: zZzZzZzZz
Amigo: zZzZzZzZz
Amigo: zZzZzZzZz
Amigo: zZzZzZzZz
Amigo: zZzZzZzZz
Amigo: zZzZzZzZz
Amigo: Me acabo de despertar
Entonces vamos a jugas :)
Amigo: *jugando*
*jugando*
Amigo: *jugando*
*jugando*
Amigo: *jugando*
*jugando*


### Ejercicio: DCChef
Cansado de las filas para usar el microondas y del precio de la comida dentro de la universidad, el DCC decidió crear su propio restaurante, y te han designado a ti para administrarlo! Decides modelar el funcionamiento de tu programa con lo que has aprendido en el curso de Programación Avanzada. Todo parece ir bien hasta que te das cuenta que se ha formado una colosal fila para entrar, ya que tu programa no permite que se tomen los pedidos mientras los cocineros están cocinando! ¿Cómo podríamos hacer que estas acciones se ejecuten a la vez? ¿Es esto el fin para el DCChef?

Para solucionar este problema, se modelaron 4 clases: ```Gerente```, ```Cocinero```, ```Garzon``` y ```Restoran```. El ```Gerente``` se encarga de tomar los pedidos, que están detallados en el archivo ```data.csv```. Las entidades de la clase ```Cocinero``` se encargan de cocinar cada pedido y las entidades de la clase ```Garzon``` entregan los platos a las mesas.

La clase ```Restoran``` tendrá las _deques_ de pedidos y de platos, que serán pasadas como argumentos al ```Gerente``` y a las entidades de ```Garzon``` y  ```Cocinero``` según sea necesario. Las _deque_ funcionan igual que las listas en el sentido que cuando le pasamos la misma _deque_ como argumento a las clases, este elemento va a estar compartido, es decir, si cualquier instancia modifica alguna de las dos _deques_, se modificará para todas las clases. En este caso esto es el comportamiento deseado, ya que queremos que los pedidos y los platos sean iguales para todos, pero se deberá manejar con ```Lock```'s.

El restorán cerrará cuando se active el ```Event``` ```platos_entregados```, que significa que ya se entregaron todos los platos. Este evento lo deben activar los garzones cuando la _deque_ de platos esté vacía, pero sólo si el ```Event``` ```platos_cocinados``` ya está activado, ya que puede que algún cocinero todavía le falte cocinar algún plato. Por su parte, este último evento es activado por los cocineros cuando la _deque_ de pedidos esta vacía, pero sólo si es ```Event``` ```pedidos_tomados``` ya está activado, ya que puede que al gerente todavía le falte tomar algún pedido. Por último, este evento es activado por el gerente cuando ya se tomaron todos los pedidos. En resumen, el restorán espera a que los garzones terminen de trabajar, quienes esperan que los cocineros terminen de trabajar, quienes esperan a que el gerente termine de trabajar.

Además, deberás completar los siguientes métodos:

- ```Gerente.__init__()```: deberás completar lo que falta en el inicializador de esta clase.
- ```Gerente.run()```: deberás abrir el archivo e ir línea por línea instanciando los pedidos como la _named tuple_ ```Pedido```, y añadiendo cada uno a la lista de pedidos, usando el ```Lock``` correspondiente. Luego de añadir cada pedido, el gerente esperará una cantidad aleatoria de segundos entre 1 y 3 (```time.sleep()```), hará el siguiente print: ```"Gerente {nombre} anotando pedido de {nombre_pedido} para mesa {mesa}"```, y al haber tomado todos los pedidos, activará el evento ```pedidos_tomados_evt```.
- ```Cocinero.__init__()```: deberás completar lo que falta en el inicializador de esta clase.
- ```Cocinero.run()```: el cocinero estará en un _loop_ donde tomará el ```Lock``` de los pedidos y revisará si hay pedidos en la _deque_. Si es que no hay pedidos en la _deque_, entonces activará el evento ```platos_cocinados_evt``` sólo si todos los pedidos ya fueron tomados, luego, independientemente de si los pedidos fueron tomados o no, pasará a la siguiente iteración (```continue```). Si es que hay pedidos en la _deque_, entonces sacará el primer pedido y se tomará de 2 a 5 segundos en cocinarlo, haciendo el siguiente print: ```"Cociner@ {nombre} cocinando {nombre_pedido} para la mesa {mesa}"```, para después tomar el ```Lock``` de platos y añadirlo a la _deque_ de platos.
- ```Garzon.__init__()```: deberás completar lo que falta en el inicializador de esta clase.
- ```Garzon.run()```: el garzón estará en un _loop_ donde tomará el ```Lock``` de los platos y revisará si hay platos en la _deque_. Si es que no hay platos en la _deque_, entonces activará el evento ```platos_entregados``` sólo si todos los platos ya fueron cocinados, luego, independientemente de si los platos fueron cocinados o no, pasará a la siguiente iteración (```continue```). Si es que hay platos en la _deque_, entonces sacará el primer plato y se tomará de 2 a 5 segundos en entregarlo, y hará el siguiente print: ```"Garzón {nombre} entregando plato {nombre_plato} a mesa {mesa}"```.
- ```Restoran.__init__()```: deberás completar lo que falta en el inicializador de esta clase.
- ```Restoran.abrir()```: esta función es la encargada de abrir el restorán, deberá instanciar a un gerente e iniciar su ```Thread```, instanciar los cocineros e iniciar sus ```Thread```'s e instanciar los garzones e iniciar sus ```Thread```'s, guardando estos últimos dos en sus listas correspondientes. Por último esperará a que se active el evento ```platos_entregados_evt``` para poder cerrar el restorán.

In [24]:
from collections import deque, namedtuple
from threading import Event, Lock, Thread
import random
import time

Pedido = namedtuple("Pedido", ["nombre", "mesa"])


class Gerente(Thread):

    def __init__(self, nombre: str, nombre_archivo: str, pedidos: deque,
                 lock_pedidos: Lock, lock_print: Lock, pedidos_tomados_evt: Event):
        # completar
        self.nombre = nombre
        self.nombre_archivo = nombre_archivo
        self.pedidos = pedidos
        self.lock_pedidos = lock_pedidos
        self.lock_print = lock_print
        self.pedidos_tomados_evt = pedidos_tomados_evt

    def run(self):
        # completar
        pass


class Cocinero(Thread):

    def __init__(self, nombre: str, pedidos: deque, platos: deque, lock_pedidos: Lock, lock_platos: Lock,
                 lock_print: Lock, pedidos_tomados_evt: Event, platos_cocinados_evt: Event):
        # completar
        self.nombre = nombre
        self.platos = platos
        self.pedidos = pedidos
        self.lock_pedidos = lock_pedidos
        self.lock_platos = lock_platos
        self.lock_print = lock_print
        self.pedidos_tomados_evt = pedidos_tomados_evt
        self.platos_cocinados_evt = platos_cocinados_evt
        self.daemon # completar

    def run(self):
        while True:
            with self.lock_pedidos:
                # completar
                time.sleep(random.randint(2, 5))
                with self.lock_platos:
                    # completar
                    pass
            time.sleep(random.randint(0, 1)) # esto es para aleatorizar el cocinero que tomará el pedido


class Garzon(Thread):

    def __init__(self, nombre: str, platos: deque, lock_platos: Lock, lock_print: Lock,
                 platos_cocinados_evt: Event, platos_entregados_evt: Event):
        # completar
        self.nombre = nombre
        self.platos = platos
        self.lock_platos = lock_platos
        self.lock_print = lock_print
        self.platos_cocinados_evt = platos_cocinados_evt
        self.platos_entregados_evt = platos_entregados_evt
        self.daemon # completar

    def run(self):
        while True:
            with self.lock_platos:
                # completar
                time.sleep(random.randint(2, 5))
                # completar
            time.sleep(random.randint(0, 1)) # esto es para aleatorizar el garzón que tomará el pedido


class Restoran:

    def __init__(self, nombre_restoran: str):
        self.nombre_restoran = nombre_restoran
        self.pedidos = deque()
        self.platos = deque()
        self.lock_pedidos = None # completar
        self.lock_platos = None # completar
        self.pedidos_tomados_evt = None # completar
        self.platos_cocinados_evt = None # completar
        self.platos_entregados_evt = None # completar
        self.garzones = []
        self.cocineros = []

    def abrir(self, nombre_archivo: str, nombre_gerente: str,
              nombres_garzones: list, nombres_cocineros: list):
        print(f"¡Buenos días! Ya abrimos el restorán {self.nombre_restoran}")
        # completar
        print(f"¡Buenas noches! Ya cerramos el restorán {self.nombre_restoran}")

In [25]:
garzones = ["Julián", "Clemente", "Diego", "Julio", "Carlos"]
cocineros = ["Hernán", "Dani", "Gatochico", "Dante", "Paqui"]
restoran = Restoran("DCChef")
restoran.abrir("data.csv", "Cata", garzones, cocineros)

¡Buenos días! Ya abrimos el restorán DCChef
¡Buenas noches! Ya cerramos el restorán DCChef
